# 0. Configuration

In [1]:
# links to shared data MovieLens
# source on kaggle: https://www.kaggle.com/code/quangnhatbui/movie-recommender/data
RATINGS_SMALL_URL = 'https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link'
MOVIES_METADATA_URL = 'https://drive.google.com/file/d/19g6-apYbZb5D-wRj4L7aYKhxS-fDM4Fb/view?usp=share_link'

# 1. Modules and functions

In [2]:
# just to make it available to download w/o SSL verification
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import scipy.sparse as sp

from itertools import islice, cycle, product

from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')


## 1. 1. Helper functions to avoid copy paste

In [3]:
def read_csv_from_gdrive(url):
    """
    gets csv data from a given url (taken from file -> share -> copy link)
    :url: example https://drive.google.com/file/d/1BlZfCLLs5A13tbNSJZ1GPkHLWQOnPlE4/view?usp=share_link
    """
    file_id = url.split('/')[-2]
    file_path = 'https://drive.google.com/uc?export=download&id=' + file_id
    data = pd.read_csv(file_path)

    return data

# 2. Main

## 2.1. Load Data

`interactions` dataset shows list of movies that users watched, along with given ratings:

In [4]:
# interactions data
interactions = read_csv_from_gdrive(RATINGS_SMALL_URL)
interactions.head()

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

`movies_metadata` dataset shows the list of movies existing on OKKO platform:

In [5]:
# information about films etc
movies_metadata = read_csv_from_gdrive(MOVIES_METADATA_URL)
movies_metadata.head(3)

adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   

     original_title                                           overview  ...  \
0         Toy Story  Led by Woody, Andy's toys live happily in his ...  ...   
1           Jumanji  When siblings Judy and Peter discover an encha...  ...   
2  Grumpier Old Men  A family wedding reignites the ancient feud be...  ...   

  release_date      revenue runtime  \
0   1995-10-30  373554033.0    81.0   
1   1995-12-15  262797249.0   104.0   
2   1995-12-22          0.0   101.0   

                                    spoken_languages    status  \
0           [{'iso_639_1': 'en', 'name': 'English'}]  Released   
1  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...  Released   
2           [{'iso_639_1': 'en', 'name': 'English'}]  Released   

                                             tagline             title  video  \
0                                                NaN         Toy Story  False   
1          Roll the dice and unleash the excitement!           Jumanji  False   
2  Still Yelling. Still Fighting. Still Ready for...  Grumpier Old Men  False   

  vote_average vote_count  
0          7.7     5415.0  
1          6.9     2413.0  
2          6.5       92.0  

[3 rows x 24 columns]

In [6]:
movies_metadata['id'] = movies_metadata['id'].astype(str)
interactions['movieId'] = interactions['movieId'].astype(str)

In [7]:
# leave only those films that intersect with each other
interactions_filtered = interactions.loc[interactions['movieId'].isin(movies_metadata['id'])]
print(interactions.shape, interactions_filtered.shape)

(100004, 4) (44989, 4)


## 2.2 Data preparation using LightFM Dataset

To use implicit kNN method `fit` we need a sparse matrix in COOrdinate format. To achieve that we will use `scipy.sparse.coo_matrix` from scipy;


In [8]:
def get_coo_matrix(
        df: pd.DataFrame, 
        user_col: str,
        item_col: str, 
        users_mapping: dict, 
        movies_mapping: dict,
        weight_col: str = None
        ):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)
    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(movies_mapping.get)
        )
    ))
    return interaction_matrix


In [9]:
# define users mapping
users_inv_mapping = dict(enumerate(interactions_filtered['userId'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)


671

In [10]:
# define movies mapping
movies_inv_mapping = dict(enumerate(interactions_filtered['movieId'].unique()))
movies_mapping = {v: k for k, v in movies_inv_mapping.items()}
len(movies_mapping)


2830

In [11]:
# defining train set on the whole interactions dataset (as HW you will have to split into test and train for evaluation)
train_mat = get_coo_matrix(
    interactions_filtered,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()


In [12]:
train_mat

<671x2830 sparse matrix of type '<class 'numpy.float32'>'
	with 44989 stored elements in Compressed Sparse Row format>

## 2.3. Model Training & Evaluation

In [`implicit`](https://pypi.org/project/implicit/), there are various models and can be groupped into:
- Item-to-Item: KNN based on various similarities - CosineRecommender, BM25Recommender, TFIDFRecommender
- implicit ALS;
- Logistic Matrix Factorization;
- Bayesian Personalized Ranking (BPR)


### 2.3.1. Train Model

In [13]:
from implicit.nearest_neighbours import (
    CosineRecommender,
    BM25Recommender,
    TFIDFRecommender
    )


Note that in item-to-item models we need to provide matrix in the form of item-user by transposing initial COO matrix user-item


In [14]:
# fit the model
cosine_model = CosineRecommender(K = 20)
cosine_model.fit(train_mat.T)


100%|██████████| 671/671 [00:00<00:00, 46448.78it/s]


### 2.3.2. Evaluate the Model

In [15]:
# let's make sense-check
top_N = 10
user_id = interactions_filtered['userId'].iloc[0]
row_id = users_mapping[user_id]
print(f'Rekko for user {user_id}, row number in matrix - {row_id}')

Rekko for user 1, row number in matrix - 0


In [16]:
# create mapper for movieId and title names
movie_name_mapper = dict(zip(movies_metadata['id'], movies_metadata['original_title']))

In [17]:
recs = cosine_model.recommend(
    row_id,
    train_mat,
    N = top_N,
    filter_already_liked_items = True
    )
recs = pd.DataFrame(recs).T.rename(columns = {0: 'col_id', 1: 'similarity'})
recs['inv_movie_id'] = recs['col_id'].astype(int)
recs['movieId'] = recs['inv_movie_id'].map(movies_inv_mapping.get)
recs['title'] = recs['movieId'].map(movie_name_mapper)
recs


col_id  similarity  inv_movie_id movieId                         title
0   653.0    0.861587           653   74458        Mere Brother Ki Dulhan
1   129.0    0.844531           129    1994       The Most Dangerous Game
2   606.0    0.654064           606    8011  Highlander III: The Sorcerer
3   294.0    0.625141           294      70           Million Dollar Baby
4   337.0    0.593856           337     170                 28 Days Later
5   648.0    0.577499           648   68954                     Longitude
6   579.0    0.571681           579    5956                        Joshua
7   399.0    0.561442           399    1088                   Whale Rider
8   278.0    0.561442           278    1584                School of Rock
9   150.0    0.557086           150    2100               The Last Castle

# TODO
- Make global train/ global test split -- train the model appropiately and predict on test set;
- Wrap up in function recommendations - lfm_recommend();
- Calculate `NDCG@10` on test set

- Make global train/ global test split -- train the model appropiately and predict on test set;

In [18]:
interactions_filtered = interactions_filtered.sort_values(by='timestamp', ascending=True) # filtered by time
split = round(len(interactions_filtered)*0.75)
train_global = interactions_filtered[:split]
test_global = interactions_filtered[split:]

In [19]:
# training 
train_mat_global = get_coo_matrix(
    train_global,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()

In [20]:
cosine_model = CosineRecommender(K = 20)
cosine_model.fit(train_mat_global.T)

100%|██████████| 671/671 [00:00<00:00, 57184.21it/s]


In [21]:
top_N = 10
user_id = train_global['userId'].iloc[0]
row_id = users_mapping[user_id]
print(f'Rekko for user {user_id}, row number in matrix - {row_id}')

Rekko for user 383, row number in matrix - 382


In [22]:
test_mat_global = get_coo_matrix(
    test_global,
    user_col = 'userId',
    item_col = 'movieId',
    users_mapping = users_mapping,
    movies_mapping = movies_mapping
    ).tocsr()

In [32]:
recs_hw = cosine_model.recommend(
    row_id,
    test_mat_global,
    N = top_N,
    filter_already_liked_items = True
    )
recs_hw = pd.DataFrame(recs_hw).T.rename(columns = {0: 'col_id', 1: 'similarity'})
recs_hw['inv_movie_id'] = recs_hw['col_id'].astype(int)
recs_hw['movieId'] = recs_hw['inv_movie_id'].map(movies_inv_mapping.get)
recs_hw['title'] = recs_hw['movieId'].map(movie_name_mapper)
recs_hw

col_id  similarity  inv_movie_id movieId  \
0   653.0    0.861587           653   74458   
1   129.0    0.844531           129    1994   
2   579.0    0.816769           579    5956   
3   460.0    0.776079           460    1895   
4   606.0    0.654064           606    8011   
5   337.0    0.593856           337     170   
6   648.0    0.577499           648   68954   
7   399.0    0.561442           399    1088   
8   278.0    0.561442           278    1584   
9   150.0    0.557086           150    2100   

                                          title  
0                        Mere Brother Ki Dulhan  
1                       The Most Dangerous Game  
2                                        Joshua  
3  Star Wars: Episode III - Revenge of the Sith  
4                  Highlander III: The Sorcerer  
5                                 28 Days Later  
6                                     Longitude  
7                                   Whale Rider  
8                                School of Rock  
9                               The Last Castle

- Wrap up in function recommendations - lfm_recommend();

In [24]:
def lfm_recommend(file,
                  row_id,
                  test,
                  N,
                  filter_already_liked_items) -> pd.DataFrame:
    
    recs = cosine_model.recommend(
        row_id,
        file,
        N,
        filter_already_liked_items)
    recs = pd.DataFrame(recs).T.rename(columns = {0: 'col_id', 1: 'similarity'})
    recs['inv_movie_id'] = recs['col_id'].astype(int)
    recs['movieId'] = recs['inv_movie_id'].map(movies_inv_mapping.get)
    recs['title'] = recs['movieId'].map(movie_name_mapper)
    return recs

- Calculate `NDCG@10` on test set

In [47]:
def ndcg_at_k(scores, k):
    best_scores = sorted(scores, reverse=True)[:k]
    best_dcg = 0
    for i in range(len(best_scores)):
        best_dcg += (2 ** best_scores[i] - 1) / np.log2(i + 2)
        
    true_scores = scores[:k]
    true_dcg = 0
    for i in range(len(true_scores)):
        true_dcg += (2 ** true_scores[i] - 1) / np.log2(i + 2)
        
    if best_dcg == 0:
        ndcg = 0
    else:
        ndcg = true_dcg / best_dcg
        
    return ndcg

print(ndcg_at_k(recs_hw.similarity, 10))

1.0
